In [151]:
from functools import wraps


class Wrap:
  def f(*args,**kwargs): # placeholder identity func
    return args,kwargs
  def pre(self):  pass
  def get(self,*args,**kwargs):
    return self.f(*args,**kwargs)
  def post(self):  pass
  def __call__(self, f):
    self.f = f
    @wraps(f)
    def _a(*args,**kwargs):
      self.pre()
      res = self.get(*args,**kwargs)
      self.post()
      return res
    return _a
  def __repr__(self):
    return str(self.__dict__)


help(Wrap())

Help on Wrap in module __main__ object:

class Wrap(builtins.object)
 |  Methods defined here:
 |  
 |  __call__(self, f)
 |      Call self as a function.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  f(*args, **kwargs)
 |  
 |  get(self, *args, **kwargs)
 |  
 |  post(self)
 |  
 |  pre(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [176]:
import time

from utils.base import log
logf=log.warning


# def timit decorator
def timit(*args,**kwargs):
  self = Wrap()
  def t0(): # sidestep lambda's no-assign constraint
    self.t0 = time.time()
  self.pre = t0
  self.post = lambda:logf('%s took %s', f.__name__, time.time()-self.t0)
  return self


# test unit
_timit = timit()
_timit.pre()
_timit.post()
_timit

# test inter
def f():  logf('f()')
# func
timit()(f)()
# deco
@timit()
def f():  logf('f()')
f()

f took 2.2172927856445312e-05
f()
f took 0.0008749961853027344
f()
f took 0.0008819103240966797


In [177]:
from collections import OrderedDict


def memo(f_k=lambda obj:obj, dict_type=OrderedDict):
  self = Wrap()
  self.seen = dict_type()
  def get(*args,**kwargs):
    key = f_k(*args,**kwargs)
    if key in self.seen:
      logf('> %s', key)
    else:
      val = self.f(*args,**kwargs)
      logf('+ %s', key)
      self.seen[key] = val
    return self.seen[key]
  self.get = get
  return self


# test unit
_memo = memo()
_memo

# test inter
import random
xs = random.choices(range(3),k=5)
mapl = lambda f,itr:list(map(f,itr))
# func
def echo(x): return x
mapl(memo()(echo), xs)
logf('')
# deco
@memo()
def echo(x): return x
mapl(echo, xs)

+ 2
+ 1
+ 0
> 1
> 0

+ 2
+ 1
+ 0
> 1
> 0


[2, 1, 0, 1, 0]